In [1]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install playwright

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.2/34.2 MB 14.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


## This web scraping methodology was sourced from VikParuchuri and his repo nba_games

In [3]:
!playwright install

140.3 Mb [                    ] 0% 0.0s140.3 Mb [                    ] 0% 69.5s140.3 Mb [                    ] 0% 27.0s140.3 Mb [                    ] 0% 95.6s140.3 Mb [                    ] 0% 44.2s140.3 Mb [                    ] 0% 26.8s140.3 Mb [                    ] 1% 19.2s140.3 Mb [                    ] 1% 19.0s140.3 Mb [                    ] 2% 13.4s140.3 Mb [=                   ] 3% 11.3s140.3 Mb [=                   ] 3% 9.4s140.3 Mb [=                   ] 4% 9.1s140.3 Mb [=                   ] 4% 8.5s140.3 Mb [=                   ] 4% 8.8s140.3 Mb [=                   ] 5% 7.8s140.3 Mb [=                   ] 6% 7.4s140.3 Mb [=                   ] 6% 7.7s140.3 Mb [=                   ] 7% 7.3s140.3 Mb [==                  ] 7% 7.0s140.3 Mb [==                  ] 8% 6.7s140.3 Mb [==                  ] 8% 6.6s140.3 Mb [==                  ] 9% 7.4s140.3 Mb [==                  ] 9% 7.0s140.3 Mb [==                  ] 10% 6.8s140.3 Mb [==                  ] 10% 6.6s140.3 Mb [==  

In [33]:
import os 
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

import pandas as pd

In [35]:
SEASONS = list(range(2016, 2023))
DATA_DIR = 'data'
STANDINGS_DIR = os.path.join(DATA_DIR, 'standings')
SCORES_DIR = os.path.join(DATA_DIR, 'scores')

In [29]:
async def get_html(url, selector, sleep=5, retries=5):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i * 2)
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html


In [9]:
season = 2016
url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
url 

'https://www.basketball-reference.com/leagues/NBA_2016_games.html'

In [10]:
html = await get_html(url, "#content .filter")

2015-16 NBA Schedule | Basketball-Reference.com


In [11]:
html

'\n\n<div class=" current">\n\t<a href="/leagues/NBA_2016_games-october.html">October</a>\n</div><div class="">\n\t<a href="/leagues/NBA_2016_games-november.html">November</a>\n</div><div class="">\n\t<a href="/leagues/NBA_2016_games-december.html">December</a>\n</div><div class="">\n\t<a href="/leagues/NBA_2016_games-january.html">January</a>\n</div><div class="">\n\t<a href="/leagues/NBA_2016_games-february.html">February</a>\n</div><div class="">\n\t<a href="/leagues/NBA_2016_games-march.html">March</a>\n</div><div class="">\n\t<a href="/leagues/NBA_2016_games-april.html">April</a>\n</div><div class="">\n\t<a href="/leagues/NBA_2016_games-may.html">May</a>\n</div><div class="">\n\t<a href="/leagues/NBA_2016_games-june.html">June</a>\n</div>'

In [12]:
soup = BeautifulSoup(html)
links = soup.find_all('a')
href = [l['href'] for l in links]

In [13]:
href

['/leagues/NBA_2016_games-october.html',
 '/leagues/NBA_2016_games-november.html',
 '/leagues/NBA_2016_games-december.html',
 '/leagues/NBA_2016_games-january.html',
 '/leagues/NBA_2016_games-february.html',
 '/leagues/NBA_2016_games-march.html',
 '/leagues/NBA_2016_games-april.html',
 '/leagues/NBA_2016_games-may.html',
 '/leagues/NBA_2016_games-june.html']

In [14]:
standings_pages = [f"https://basketball-reference.com{h}" for h in href]
standings_pages

['https://basketball-reference.com/leagues/NBA_2016_games-october.html',
 'https://basketball-reference.com/leagues/NBA_2016_games-november.html',
 'https://basketball-reference.com/leagues/NBA_2016_games-december.html',
 'https://basketball-reference.com/leagues/NBA_2016_games-january.html',
 'https://basketball-reference.com/leagues/NBA_2016_games-february.html',
 'https://basketball-reference.com/leagues/NBA_2016_games-march.html',
 'https://basketball-reference.com/leagues/NBA_2016_games-april.html',
 'https://basketball-reference.com/leagues/NBA_2016_games-may.html',
 'https://basketball-reference.com/leagues/NBA_2016_games-june.html']

In [30]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    standings_pages = [f"https://www.basketball-reference.com{l['href']}" for l in links]
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [ ]:
for season in SEASONS:
    await scrape_season(season)

In [32]:
standings_files = os.listdir(STANDINGS_DIR)
standings_files

['NBA_2022_games-october.html',
 'NBA_2021_games-june.html',
 'NBA_2020_games-march.html',
 'NBA_2020_games-september.html',
 'NBA_2020_games-january.html',
 'NBA_2020_games-august.html',
 'NBA_2019_games-april.html',
 'NBA_2022_games-may.html',
 '.DS_Store',
 'NBA_2019_games-february.html',
 'NBA_2018_games-february.html',
 'NBA_2016_games-april.html',
 'NBA_2021_games-march.html',
 'NBA_2018_games-january.html',
 'NBA_2017_games-february.html',
 'NBA_2016_games-february.html',
 'NBA_2017_games-october.html',
 'NBA_2018_games-april.html',
 'NBA_2020_games-december.html',
 'NBA_2019_games-october.html',
 'NBA_2020_games-november.html',
 'NBA_2021_games-may.html',
 'NBA_2021_games-december.html',
 'NBA_2022_games-april.html',
 'NBA_2020_games-october-2019.html',
 'NBA_2022_games-december.html',
 'NBA_2017_games-april.html',
 'NBA_2022_games-november.html',
 'NBA_2016_games-january.html',
 'NBA_2018_games-october.html',
 'NBA_2017_games-march.html',
 'NBA_2021_games-february.html',
 'NBA

In [46]:
async def scrape_game(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get('href') for l in links]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in hrefs if l and "boxscore" in l and '.html' in l]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(html)

In [ ]:
for season in SEASONS:
    files = [s for s in standings_files if str(season) in s]
    
    for f in files:
        filepath = os.path.join(STANDINGS_DIR, f)
        
        await scrape_game(filepath)

In [36]:
boxscores = os.listdir(SCORES_DIR)
boxscores = [os.path.join(SCORES_DIR, f) for f in boxscores if f.endswith(".html")]
boxscores

['data/scores/201512090DAL.html',
 'data/scores/201710180SAS.html',
 'data/scores/202104300MEM.html',
 'data/scores/201703080MIN.html',
 'data/scores/201903260DAL.html',
 'data/scores/201511270CHO.html',
 'data/scores/202201230MIN.html',
 'data/scores/202001160NOP.html',
 'data/scores/202203180PHO.html',
 'data/scores/202102190ORL.html',
 'data/scores/201611030MIL.html',
 'data/scores/201911180HOU.html',
 'data/scores/201712020BRK.html',
 'data/scores/202111240GSW.html',
 'data/scores/201701260OKC.html',
 'data/scores/201602050ORL.html',
 'data/scores/202103030TOR.html',
 'data/scores/202103100DAL.html',
 'data/scores/202103280LAL.html',
 'data/scores/201801200NOP.html',
 'data/scores/201603110UTA.html',
 'data/scores/201711240BOS.html',
 'data/scores/202008220POR.html',
 'data/scores/201904070MEM.html',
 'data/scores/201602260PHI.html',
 'data/scores/202102120ATL.html',
 'data/scores/201604190SAS.html',
 'data/scores/201604060LAL.html',
 'data/scores/202202280ORL.html',
 'data/scores/

In [47]:
def parse_html(box_score):
    with open(box_score) as f:
        html = f.read()

    soup = BeautifulSoup(html)
    [s.decompose() for s in soup.select("tr.over_header")]
    [s.decompose() for s in soup.select("tr.thead")]
    return soup

In [48]:
def season_info(soup):
    nav = soup.select("#bottom_nav_container")[0]
    hrefs = [a["href"] for a in nav.find_all('a')]
    season = os.path.basename(hrefs[1]).split("_")[0]
    return season

In [49]:
def scoreline(soup):
    line_score = pd.read_html(str(soup), attrs = {'id': 'line_score'})[0]
    cols = list(line_score.columns)
    cols[0] = "team"
    cols[-1] = "total"
    line_score.columns = cols
    
    line_score = line_score[["team", "total"]]
    
    return line_score

In [50]:
def stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs = {'id': f'box-{team}-game-{stat}'}, index_col=0)[0]
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

In [ ]:
import warnings
warnings.filterwarnings('ignore')

games = []
base_cols = None
for box_score in boxscores:
    soup = parse_html(box_score)

    scorelines = scoreline(soup)
    teams = list(scorelines["team"])

    summaries = []
    for team in teams:
        basic = stats(soup, team, "basic")
        advanced = stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1].max(), advanced.iloc[:-1].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals, maxes])
        
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]
        
        summary = summary[base_cols]
        
        summaries.append(summary)
    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, scorelines], axis=1)

    game["home"] = [0,1]

    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)
    full_game["season"] = season_info(soup)
    
    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")
    
    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games.append(full_game)
    
    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(boxscores)}")

In [ ]:
games = pd.concat(games, ignore_index=True)
games

In [ ]:
games.to_csv("games.csv", index=False)

## Get 2024 Season Data for Simulation

In [57]:
await scrape_season(2023)

2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com


In [55]:
files = [s for s in standings_files if str(2024) in s]
for f in files:
    print(f)
    filepath = os.path.join(STANDINGS_DIR, f)
    await scrape_game(filepath)

In [59]:
await scrape_game("data/standings/NBA_2024_games-october.html")

Lakers vs Nuggets, October 24, 2023 | Basketball-Reference.com
Suns vs Warriors, October 24, 2023 | Basketball-Reference.com
Rockets vs Magic, October 25, 2023 | Basketball-Reference.com
Celtics vs Knicks, October 25, 2023 | Basketball-Reference.com
Wizards vs Pacers, October 25, 2023 | Basketball-Reference.com
Hawks vs Hornets, October 25, 2023 | Basketball-Reference.com
Pistons vs Heat, October 25, 2023 | Basketball-Reference.com
Timberwolves vs Raptors, October 25, 2023 | Basketball-Reference.com
Cavaliers vs Nets, October 25, 2023 | Basketball-Reference.com
Pelicans vs Grizzlies, October 25, 2023 | Basketball-Reference.com
Thunder vs Bulls, October 25, 2023 | Basketball-Reference.com
Kings vs Jazz, October 25, 2023 | Basketball-Reference.com
Mavericks vs Spurs, October 25, 2023 | Basketball-Reference.com
Trail Blazers vs Clippers, October 25, 2023 | Basketball-Reference.com
76ers vs Bucks, October 26, 2023 | Basketball-Reference.com
Suns vs Lakers, October 26, 2023 | Basketball-Ref

In [60]:
boxscores_2023 = [f for f in os.listdir(SCORES_DIR) if f.startswith("2023")]
boxscores_2023


['202310280CLE.html',
 '202310250IND.html',
 '202310270ATL.html',
 '202310300MIL.html',
 '202310250CHO.html',
 '202310270DAL.html',
 '202310290HOU.html',
 '202310280MIN.html',
 '202310250LAC.html',
 '202310290OKC.html',
 '202310260MIL.html',
 '202310300ATL.html',
 '202310280TOR.html',
 '202310270UTA.html',
 '202310300NOP.html',
 '202310240GSW.html',
 '202310250SAS.html',
 '202310290LAC.html',
 '202310300LAL.html',
 '202310310PHO.html',
 '202310270SAC.html',
 '202310240DEN.html',
 '202310250MEM.html',
 '202310250TOR.html',
 '202310280WAS.html',
 '202310250BRK.html',
 '202310250CHI.html',
 '202310260LAL.html',
 '202310270POR.html',
 '202310300CHO.html',
 '202310290SAC.html',
 '202310300IND.html',
 '202310250UTA.html',
 '202310250ORL.html',
 '202310280PHO.html',
 '202310310LAC.html',
 '202310250MIA.html',
 '202310270CHO.html',
 '202310300WAS.html',
 '202310280DET.html',
 '202310300OKC.html',
 '202310270CLE.html',
 '202310300TOR.html',
 '202310300MEM.html',
 '202310300DEN.html',
 '20231025

In [62]:
import warnings
warnings.filterwarnings('ignore')

games_2023 = []
base_cols = None
for box_score in boxscores_2023:
    soup = parse_html('data/scores/' + box_score)

    scorelines = scoreline(soup)
    teams = list(scorelines["team"])

    summaries = []
    for team in teams:
        basic = stats(soup, team, "basic")
        advanced = stats(soup, team, "advanced")

        totals = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
        totals.index = totals.index.str.lower()

        maxes = pd.concat([basic.iloc[:-1].max(), advanced.iloc[:-1].max()])
        maxes.index = maxes.index.str.lower() + "_max"

        summary = pd.concat([totals, maxes])
        
        if base_cols is None:
            base_cols = list(summary.index.drop_duplicates(keep="first"))
            base_cols = [b for b in base_cols if "bpm" not in b]
        
        summary = summary[base_cols]
        
        summaries.append(summary)
    summary = pd.concat(summaries, axis=1).T

    game = pd.concat([summary, scorelines], axis=1)

    game["home"] = [0,1]

    game_opp = game.iloc[::-1].reset_index()
    game_opp.columns += "_opp"

    full_game = pd.concat([game, game_opp], axis=1)
    full_game["season"] = season_info(soup)
    
    full_game["date"] = os.path.basename(box_score)[:8]
    full_game["date"] = pd.to_datetime(full_game["date"], format="%Y%m%d")
    
    full_game["won"] = full_game["total"] > full_game["total_opp"]
    games_2023.append(full_game)
    
    if len(games) % 100 == 0:
        print(f"{len(games)} / {len(boxscores)}")

Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed


In [64]:
games_2023_pd = pd.concat(games_2023, ignore_index=True)
games_2023_pd

,mp,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,...,tov%_max_opp,usg%_max_opp,ortg_max_opp,drtg_max_opp,team_opp,total_opp,home_opp,season,date,won
0,240.0,240.0,48.0,98.0,0.490,15.0,38.0,0.395,14.0,17.0,...,9.1,35.5,228.0,136.0,CLE,113,1,2024,2023-10-28,True
1,240.0,240.0,43.0,99.0,0.434,12.0,31.0,0.387,15.0,24.0,...,16.7,29.5,175.0,127.0,IND,125,0,2024,2023-10-28,False
2,240.0,240.0,44.0,96.0,0.458,9.0,24.0,0.375,23.0,29.0,...,18.4,47.6,211.0,119.0,IND,143,1,2024,2023-10-25,False
3,240.0,240.0,56.0,107.0,0.523,20.0,43.0,0.465,11.0,12.0,...,50.0,36.0,174.0,137.0,WAS,120,0,2024,2023-10-25,True
4,240.0,240.0,43.0,90.0,0.478,20.0,44.0,0.455,20.0,28.0,...,24.4,31.4,178.0,127.0,ATL,120,1,2024,2023-10-27,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,240.0,240.0,39.0,86.0,0.453,16.0,44.0,0.364,16.0,20.0,...,66.7,35.7,232.0,118.0,ATL,127,0,2024,2023-10-29,False
104,265.0,265.0,39.0,84.0,0.464,9.0,36.0,0.250,16.0,25.0,...,33.3,30.4,151.0,103.0,CHI,104,1,2024,2023-10-27,False
105,265.0,265.0,37.0,99.0,0.374,8.0,39.0,0.205,22.0,28.0,...,37.5,27.0,140.0,105.0,TOR,103,0,2024,2023-10-27,True
106,265.0,265.0,52.0,100.0,0.520,8.0,32.0,0.250,10.0,20.0,...,30.0,36.5,223.0,119.0,SAS,126,1,2024,2023-10-27,False


In [65]:
games_2023_pd.to_csv("games_2023.csv", index=False)